In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import subprocess
import os
import sys
sys.path.append('/usr/workspace/vanover1/approx-llvm/approx')
from approx_modules import approx

from sklearn.ensemble import ExtraTreesRegressor

In [3]:
# need to use /usr/tce/bin/git binary because the default git version grabbed by this subprocess is too early for the --show-superproject-working-tree flag
REPO_ROOT = subprocess.check_output("/usr/tce/bin/git rev-parse --show-superproject-working-tree --show-toplevel | head -1", shell=True).strip().decode()
N_KERNELS = 4
KERNEL_NAMES = ["s1", "t1", "t2", "t3"]

In [4]:
def run_arclength(endpoint):
    
    # compile and run arclength instrumented with HPAC directives
    subprocess.check_call(f"rm -f test.h5 && source {REPO_ROOT}/scripts/activate_env.sh && make arclength-hpac && ./arclength-hpac {endpoint}", shell=True)
    
    # open database
    approxDataProfile = approx.approxApplication("./test.h5")
    
    # get output
    Y = approxDataProfile.getApplicationOutput()['arclength'][0]

    # get aggregated kernel outputs
    kernel_outputs = []
    for kernel_name in KERNEL_NAMES:
        kernel_outputs.append(approxDataProfile[kernel_name].Y().mean())
    
    return kernel_outputs, Y

In [5]:
# define neighborhood for arclength analysis
center = np.pi
eps = np.pi
n_sample = 2**6

# uniform sample of n_sample endpoints from the neighborhood
XX = np.linspace(center-eps, center+eps, n_sample)

In [6]:
try:
    with open(f"XX_{n_sample}n.npy", "rb") as f:
        kernel_outs = np.load(f)
    with open(f"YY_{n_sample}n.npy", "rb") as f:
        YY = np.load(f)
        
except FileNotFoundError:

    # run arclength
    kernel_outs = []
    YY = []
    for X in XX:
        temp = run_arclength(X)
        kernel_outs.append(temp[0])
        YY.append(temp[1])
    kernel_outs = np.array(kernel_outs)
    YY = np.array(YY)

    with open(f"XX_{n_sample}n.npy", "wb") as f:
        np.save(f, kernel_outs)
    with open(f"YY_{n_sample}n.npy", "wb") as f:
        np.save(f, YY)

In [7]:
# train/test split
split_proportion = 0.8
split_point = int(n_sample * split_proportion)
XX_train, XX_test = kernel_outs[:split_point], kernel_outs[split_point:]
YY_train, YY_test = YY[:split_point], YY[split_point:]

print(f"Training with {split_point} samples.")

# analyze kernel output sensitivity
et = ExtraTreesRegressor(n_estimators=100,
                                    criterion="mse",
                                    max_features=int(round(XX_train.shape[1] / 3)),
                                    max_depth=8,
                                    min_samples_split=2,
                                    min_samples_leaf=max(1, int(round(np.sqrt(XX_train.shape[0]) / np.sqrt(1000)))),
                                    max_leaf_nodes=None,
                                    #bootstrap=True,
                                    #oob_score=True,
                        )#random_state=1)

et.fit(XX_train, YY_train)
print(f"Score: {et.score(XX_test, YY_test)}")

Si = [(KERNEL_NAMES[k_no], et.feature_importances_[k_no]) for k_no in range(N_KERNELS)]
Si.sort(key = lambda x : x[1])
print(f"Sensitivities: {Si}")

Training with 51 samples.
Score: -2.695859339632184
Sensitivities: [('s1', 0.22741495042636725), ('t1', 0.24804540201766748), ('t2', 0.25290835124607747), ('t3', 0.27163129630988797)]


/usr/tce/packages/python/python-3.7.2/lib/python3.7/site-packages/sklearn/ensemble/base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


## Compare to:
```
=== BEGIN ADAPT REPORT ===
15000004 total independent/intermediate variables
1 dependent variables
Mixed-precision recommendation:
  Replace variable d2      max error introduced: 0.000000e+00  count: 6000000     totalerr: 0.000000e+00
  Replace variable d1      max error introduced: 0.000000e+00  count: 1           totalerr: 0.000000e+00
  DO NOT replace   t2      max error introduced: 8.623994e-08  count: 1000000     totalerr: 8.623994e-08
  DO NOT replace   t1      max error introduced: 8.623994e-08  count: 1000001     totalerr: 1.724799e-07
  DO NOT replace   h       max error introduced: 5.401810e-07  count: 1           totalerr: 7.126609e-07
  DO NOT replace   t3      max error introduced: 9.462050e-07  count: 6000000     totalerr: 1.658866e-06
  DO NOT replace   s1      max error introduced: 6.334703e-06  count: 1000001     totalerr: 7.993569e-06
=== END ADAPT REPORT ===
Arclength - SUCCESSFUL!
```

In [8]:
# prepare data for shaff analysis
rows = []
for n in range(n_sample):
    row = {}
    for i, kernel_name in enumerate(KERNEL_NAMES):
        row[kernel_name] = kernel_outs[n][i]
    row["Y"] = YY[n]
    rows.append(row)
df = pd.DataFrame.from_dict(rows)
df = df[["Y"] + KERNEL_NAMES]
df.to_csv(f"df_{split_point}n.csv", index=False)

### From 5 SHAFF analysis runs with 204 samples:
```
[1] "s1" "t1" "t2" "t3"
[1] 0.2657836 0.2437431 0.2235744 0.2669184
[1] 0.2193727 0.2534944 0.2608640 0.2662938
[1] 0.2128173 0.2503306 0.2611730 0.2756917
[1] 0.2670222 0.2595150 0.2194171 0.2540675
[1] 0.2468810 0.2581577 0.2444400 0.2505256
```